### ISOMAP

In [4]:
import time
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.neighbors import NearestNeighbors

In [23]:
def run_floyd_warshall(graph, data):
    n = graph.shape[0]
    dist = np.full((n, n), np.inf)

    for i in range(n):
        for j in range(n):
            if i == j:
                    dist[i, j] = 0

            if graph[i, j] == 1:
                dist[i, j] = np.linalg.norm(data.iloc[i].values - data.iloc[j].values)

    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i, j] > dist[i, k] + dist[k, j]:
                    dist[i, j] = dist[i, k] + dist[k, j]

    return dist

In [100]:
def my_ISOMAP(data, dim=None, k=10):
    nn = NearestNeighbors(n_neighbors=k).fit(data)
    nn_graph = nn.kneighbors_graph(data).toarray()
    print(nn_graph)

    print(1)

    dist = run_floyd_warshall(nn_graph, data)

    print(2)

    D2 = dist ** 2
    n = D2.shape[0]
    H = np.eye(n) - np.ones((n, n)) / n
    G = -0.5 * H @ D2 @ H


    print(3)

    eigenvals, eigenvecs = np.linalg.eig(G)
    eigenvals = np.real(eigenvals)

    # sort eigenvalues and eigenvectors
    sorted_indices = np.argsort(eigenvals)[::-1]
    sorted_eigenvals = eigenvals[sorted_indices]
    sorted_eigenvecs = eigenvecs[:, sorted_indices]

    if dim is None:
        px.scatter(x=np.arange(1, len(sorted_eigenvals)+1), y=sorted_eigenvals, title="Eigenvalues", labels={"x": "Index", "y": "Eigenvalue"}, width=800, height=800).show()

        time.sleep(2)
        dim = int(float(input("Enter the number of dimensions to reduce to: ")))

    # select the top d eigenvectors
    selected_eigenvecs =  sorted_eigenvecs[:, :dim]

    # project into the d dimension
    data_iso = np.real(selected_eigenvecs * np.sqrt(sorted_eigenvals[:dim]))

    return data_iso, dim, selected_eigenvecs

In [108]:
# Swiss roll dataset
def swiss_roll_noised(n):
    """
    Parameters:
    n: int
        Number of points to generate"""

    data = np.zeros((n,3))
    phi = np.random.uniform(low=1.5*np.pi, high=4.5*np.pi, size=n)
    psi = np.random.uniform(0,10,n)

    data[:,0]=phi*np.cos(phi)+np.random.normal(0, .5) #x coordinate
    data[:,1]=phi*np.sin(phi)+np.random.normal(0, .5) #y coordinate
    data[:,2]=psi+np.random.normal(0, .5) #z coordinate
    return data

data = swiss_roll_noised(300)
data = pd.DataFrame(data, columns=["X1", "X2", "X3"])

colors = np.linalg.norm(data[["X1", "X2", "X3"]].values, axis=1)
fig = px.scatter_3d(x=data["X1"], y=data["X2"], z=data["X3"], title="Swiss Roll Dataset", color=colors, opacity=0.7, width=600, height=600)
fig.update_traces(marker=dict(size=2))
fig.show()

In [102]:
data_isomap, dim, eigenvecs = my_ISOMAP(data, dim=2)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
1
2
3


In [88]:
px.scatter(x=data_isomap[:, 0], y=data_isomap[:, 1], title=f"ISOMAP with {dim} dimensions", width=600, height=600).show()

In [113]:
from sklearn.manifold import Isomap

model = Isomap(n_neighbors=10, n_components=2)
data_sklearn = model.fit_transform(data)
colors_iso = np.linalg.norm(data_sklearn, axis=1)

In [118]:
fig = px.scatter(x=data_sklearn[:, 0], y=data_sklearn[:, 1], title=f"ISOMAP with {dim} dimensions (sklearn)", color=colors_iso, width=1000, height=1000)
fig.update_traces(marker=dict(size=1))
# add the starting data points
fig.add_scatter(
    x=data_isomap[:, 0],
    y=data_isomap[:, 1],
    mode='markers',
    marker=dict(color=colors, size=5),
    name='My ISOMAP'
)

fig.show()